# Script for Flood detection on cloud-masked Sentinel-2 data

*******************************************************************************************

## Disclaimer
This product has been derived automatically without validation data. All geographic information has limitations due to the scale, resolution, date and interpretation of the original source materials. No liability concerning the content or the use thereof is assumed by the producer.

Script created by: Geospatial Operations Support Team, The World Bank, February 2021.
*******************************************************************************************

## Applied for South Sudan flood period June - December 2020

More than 1 million people across half of South Sudan have been
affected by devastating flooding since July 2020. In response to the
floods, funding from the UN’s Central Emergency Response Fund
(CERF) and the South Sudan Humanitarian Fund (SSHF) helped
kickstart the humanitarian response to people’s needs. Combined,
the two funds allocated nearly US \$20 million to humanitarian
partners to implement life-saving assistance to 360,000 most
vulnerable people. The SSHF’s second Reserve Allocation provided
\$9.7 million for frontline response and prioritized life-saving assistance in the areas of food security and livelihoods, health, nutrition,
protection, shelter and non-food items, and water, sanitation and
hygiene. The CERF Rapid Response allocation of \$9.7 million filled
gaps in core humanitarian pipelines and enabled frontline responders
to scale up assistance to people in need. The CERF also financed
community-based initiatives aimed at reducing the impact of floods
on communities, such as repairing broken dykes (source OCHA: https://reliefweb.int/sites/reliefweb.int/files/resources/ss_20201125_south_sudan_cerf_sshf_complementarities_floods.pdf )


## Method Applied
### Optical data dervied flood mapping using the Modified Normalized Difference Water Index approach
This script also generates flood extent map using the Modified Normalised Difference Water Index
  approach (MNDWI) similar to the one defined in Rokni et al (2014) https://doi:10.3390/rs6054173

For that computation, initially performs a cloud mask using a cloud/shadow masking which requires much more processing resources/time compared to cloud mask using the bitmasks from Sentinel-2 Level-2 products, providing more accurate masks of clouds/shadows.


In [ ]:
#############################################################
# Connecting to Google Earth Engine
#############################################################

import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=v-e8eQ443EWys93ugw8ZEwPbCEIT8zE6o4Oscx3z20E&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g4_NUgmoc83EhMLj6Gr1wop0yYsqbFfijE4eNPPZDcjjln8e4loiq0

Successfully saved authorization token.


In [ ]:
#############################################################
#            SELECT YOUR OWN STUDY AREA   
#############################################################
# - The Area of Interest can be defined with the different methods:
#   1) defining country name
        countryname='South Sudan';
        countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017');
        aoi = countries.filter(ee.Filter.eq('country_na', countryname));
#   2) importing shapefile from Assets collection in GEE   (modify line below)
#      aoi=ee.FeatureCollection(users/josemanueldelgadoblasco/Selected_AOI)
AOI = ee.Geometry(aoi.geometry());

#############################################################
#           Defining flood period
#############################################################
# Flood start and end dates (YTYY-MM-DD format)
START_DATE = '2020-06-01';
END_DATE = '2020-12-31';

#############################################################
#           Defining water threshold applied to the MNDWI 
#############################################################
# MNDWI Threshold  (all above that threshold will be considered as water)
MNDWI_THRES=0.2

In [ ]:
#@title Cloudmask functions definition
#############################################################
## Defining all functions
###############################################################
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)
    #return img.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

In [ ]:
#@title cloudmask parameters definition
#############################################################
# parameters for cloudmask algorithm
#############################################################
CLOUD_FILTER = 80
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100

In [ ]:
#####################################################################################
## Computing maps for S2

s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE)
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())
b3_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                              .map(apply_cld_shdw_mask)
                              .select(['B3']).median())
b11_p10 = (s2_sr_cld_col.map(add_cld_shdw_mask)
                              .map(apply_cld_shdw_mask)
                              .select('B11').reduce(ee.Reducer.percentile([10])).rename('B11'))
mndwi = (b3_sr_median.subtract(b11_p10)).divide(b3_sr_median.add(b11_p10))

#####################################################################################
# Estimate flood classifying mndwi map usig threshold 
flood=mndwi.gt(MNDWI_THRES)

#####################################################################################
## Filtering / Refining based on permanent water layer, slope and connectivity
#####################################################################################

swater = ee.Image('JRC/GSW1_0/GlobalSurfaceWater').select('seasonality');
swater_mask = swater.gte(10).updateMask(swater.gte(10));
flooded_mask = flood.where(swater_mask,0);
flooded = flooded_mask.updateMask(flooded_mask);
#//// This operation reduces noise of the flood extent product 
connections = flooded.connectedPixelCount();    
flooded = flooded.updateMask(connections.gte(8));
# Mask out areas with more than 5 percent slope using a Digital Elevation Model 
DEM = ee.Image('WWF/HydroSHEDS/03VFDEM');
terrain = ee.Algorithms.Terrain(DEM);
slope = terrain.select('slope');

## Final layer to export
flooded = flooded.updateMask(slope.lt(5));

In [ ]:
#@title Exporting flood map to Google Drive
# Export to Drive
task = ee.batch.Export.image.toDrive(**{
    'image': flooded,
    'description': 'SouthSudan_flood_S2',
    'folder':'WB_SouthSudan_S2',
    'scale': 20,
    'region': AOI.getInfo()['coordinates'],
    'maxPixels': 1e12
})
task.start()
print('Task export launched...')